In [1]:
# Run this one if using google colab
from google.colab import drive
drive.mount('/content/gdrive')

import sys
sys.path.append("/content/xray_code/python_code")



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,40 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/gdrive/My Drive/xray/                                        14 files, 32514299019 bytes (31 GiB)
  0%    
Extracting archiv

In [ ]:
%load_ext autoreload
%autoreload 2

from utils import *
from ml_utils import XrayTF
from tqdm import tqdm_notebook as tqdm
c
import matplotlib.pyplot as plt
import numpy as np
import os

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\mnavaltx\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\users\mnavaltx\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\users\mnavaltx\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "c:\users\mnavaltx\appdata\local\programs\python\python37\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "c:\users\mnavaltx\appdata\local\programs\python\python37\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportEr

TypeError: can only concatenate str (not "list") to str

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Error in callback <bound method AutoreloadMagics.post_execute_hook of <autoreload.AutoreloadMagics object at 0x000002085038C048>> (for post_execute):
Traceback (most recent call last):
  File "c:\users\mnavaltx\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\users\mnavaltx\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\users\mnavaltx\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "c:\users\mnavaltx\appdata\local\programs\python\python37\lib\imp.py", line 242, in load_module
    return load_dynamic(name, f

In [0]:
GENDER   = "M"
POSITION = "PA"
csv_file = "normalized_xray_data_with_no_finding.csv"
csv_file_full_path = f'/content/xray_code/sheet/{csv_file}'
IMAGE_SIZE = 1000
BATCH_SIZE = 32
LABEL_QUOTA = 10000 #10k

# take existing if it exist
if os.path.exists(csv_file_full_path):
    df = pd.read_csv(csv_file_full_path)
    xray_class = XrayTF(df, IMAGE_SIZE, BATCH_SIZE)
else:
    df = get_data_sheet()
    df = normalize_data_frame(df)
    xray_class = XrayTF(df, IMAGE_SIZE, BATCH_SIZE)
    xray_class.prepend_image_full_path()
    xray_class.df.to_csv(csv_file_full_path, index=False)

xray_class.df.head()

In [0]:
# let's balance the data
"""
xray_class.balance_all_labels()
"""
xray_class.prune_based_on_quota(LABEL_QUOTA)
xray_class.balance_all_labels()

# Verify how balanced our data is
print(xray_class.df.labels.value_counts())
print(xray_class.df["Patient Gender"].value_counts())
print(xray_class.df["View Position"].value_counts())

In [0]:
# decide which one we want to take in terms of relating to other features.

"""

"""
xray_class.df = xray_class.df[xray_class.df["Patient Gender"] == GENDER]
xray_class.df = xray_class.df[xray_class.df["View Position"] == POSITION]

In [0]:
# Split them into training and validation using NUM_IMAGES 
train_df, valid_df = xray_class.get_test_train_split_data(len(xray_class.df))

In [0]:
# Create training and validation data batches using ImageDataGenerator
"""
train_data = xray_class.create_data_batches(X_train, y_train)
val_data = xray_class.create_data_batches(X_val, y_val, valid_data=True)
train_data.element_spec, val_data.element_spec
"""

train_data = xray_class.generate_image(train_df)
val_data   = xray_class.generate_image(valid_df)

In [0]:
"""
train_images, train_labels = next(train_data.as_numpy_iterator())
xray_class.show_25_images(train_images, train_labels)
"""

train_images, train_labels = next(train_data)
xray_class.show_25_images(train_images, train_labels)

In [0]:
model = xray_class.train_model(5, train_data, val_data) #(number of epoc, early stopping)
xray_class.save_model(model, "all_sickness")

In [0]:
# Make predictions on the validation data (not used to train on)
predictions = model.predict(val_data, verbose=1) # verbose shows us how long there is to go

In [0]:
# Let's check a few predictions and their different values
val_images, val_labels = next(val_data)

i_multiplier = 0
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(5*2*num_cols, 5*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  xray_class.plot_pred(prediction_probabilities=predictions,
            labels=val_labels,
            images=val_images,
            n=i+i_multiplier)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  xray_class.plot_pred_conf(prediction_probabilities=predictions,
                labels=val_labels,
                n=i+i_multiplier)
plt.tight_layout(h_pad=1.0)
plt.show()